# Agent Birth Rate Calibration

## Imports

In [3]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sys import path
import time
from tqdm import tqdm

%matplotlib inline

In [ ]:
path.append('../src/model/')
from stationsim_gcs import Model

## Paths and Directories

## Functions

In [ ]:
def get_number_pedestrians_in_frame(df, frame):
    tdf = df.loc[df['frame']==frame, :]
    return len(tdf['pedestrianID'].unique())

In [ ]:
def __get_error(observed, model, variable):
    return observed[variable] - model[variable]

In [ ]:
def get_error(observed_results, model_results):
    n_max_peds = __get_error(observed_results, model_results, 'n_max_active_pop')
    t_max_peds = __get_error(observed_results, model_results, 't_max_active_pop')
    t_max = __get_error(observed_results, model_results, 't_max')
    
    results = {'n_max_active_pop': n_max_peds,
               't_max_active_pop': t_max_peds,
               't_max': t_max,
               'rate': model_results['rate']}
    return results